In [31]:
import pandas as pd
import numpy as np
import os
from shapely.geometry import Point, Polygon
import geopandas as gpd
from shapely.ops import nearest_points
import pyproj

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)

from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Lasso, Ridge, LinearRegression, ElasticNet

import seaborn as sns
import matplotlib.pyplot as plt

cwd =os.getcwd()

In [32]:
background_data = pd.read_csv('final_data2015.csv').drop(columns='geometry')
geodata = gpd.read_file(cwd+'\scrape_geodata\geodata\dagi_10m_nohist_l1.afstemningsomraade\\afstemningsomraade.shp', driver = 'ESRI Shapefile')
geodata = geodata[['objectid', 'geometry']]
geodata['objectid'] = geodata['objectid'].astype(int)
final_gdf = geodata.merge(background_data, on='objectid', how='right')

final_gdf['left_share_norm']=final_gdf['left_share']/(final_gdf['left_share']+final_gdf['right_share'])
final_gdf['pop_density_log']=np.log(final_gdf['pop_density'])
final_gdf['disindk_log']=np.log(final_gdf['disindk'])
final_gdf['salgspris_log']=np.log(final_gdf['salgspris'])

In [33]:
X = final_gdf[['karakter', 'salgspris_log', 'kriminelitet', 'lavindk',
               'langledig', 'skilsmisser', 'andel_indv', 'pop_density_log',
               'disindk_log']]
y = final_gdf['left_share_norm']

# splitting into development (2/3) and test data (1/3)
X_dev, X_test, y_dev, y_test = train_test_split(X, y, test_size=1/3, random_state=1)
# splitting development into train (1/3) and validation (1/3)
X_train, X_val, y_train, y_val = train_test_split(X_dev, y_dev, test_size=1/2, random_state=1)

## Linear regression model

In [34]:
pipe_lr = make_pipeline(PolynomialFeatures(include_bias=True), 
                           StandardScaler(),
                            LinearRegression())
pipe_lr.fit(X_train, y_train)
y_pred = pipe_lr.predict(X_val)
mse(y_pred, y_val)

0.004162525363956261

## Lasso model

In [35]:
# splitting into development (2/3) and test data (1/3)
X_dev, X_test, y_dev, y_test = train_test_split(X, y, test_size=1/3, random_state=1)
# splitting development into train (1/3) and validation (1/3)
X_train, X_val, y_train, y_val = train_test_split(X_dev, y_dev, test_size=1/2, random_state=1)

perform = []
lambdas = np.logspace(-6, -4, 100)
for lambda_ in lambdas:
    pipe_lasso = make_pipeline(PolynomialFeatures(include_bias=True), 
                               StandardScaler(),
                               Lasso(alpha=lambda_, random_state=1, fit_intercept=True))
    pipe_lasso.fit(X_train, y_train)
    y_pred = pipe_lasso.predict(X_val)
    perform.append(mse(y_pred, y_val))
    
hyperparam_perform = pd.Series(perform,index=lambdas)
optimal = hyperparam_perform.nsmallest(1)
print('Optimal lambda:', optimal.index[0])
print('Validation MSE: %.8f' % optimal.values[0])
lambda_lasso = optimal.index[0] # Sofie Add on

Optimal lambda: 4.328761281083062e-05
Validation MSE: 0.00392724


In [36]:
"""plot_df = pd.DataFrame(hyperparam_perform).reset_index()
plot_df['index'] = np.log10(plot_df['index'])
plot_df['x_axis'] = plot_df['index']
sns.lineplot(data=plot_df, x='x_axis', y=0)"""

"plot_df = pd.DataFrame(hyperparam_perform).reset_index()\nplot_df['index'] = np.log10(plot_df['index'])\nplot_df['x_axis'] = plot_df['index']\nsns.lineplot(data=plot_df, x='x_axis', y=0)"

In [37]:
# Define preferable model
pipe_lasso = make_pipeline(PolynomialFeatures(include_bias = True),
                           StandardScaler(),
                           Lasso(alpha = lambda_lasso, random_state = 1, fit_intercept = True))

pipe_lasso.fit(X_dev, y_dev) 
"""
errors = pipe_lasso.predict(X_test)-y_test
error_gdf = final_gdf.merge(pd.DataFrame(errors), left_index=True, right_index=True, how='left')
error_gdf.plot(column='left_share_norm_y', legend=True)
"""
print('Lasso MSE on test data:', mse(pipe_lasso.predict(X_test),y_test))

Lasso MSE on test data: 0.00451162013648022


## Ridge

In [38]:
# splitting into development (2/3) and test data (1/3)
X_dev, X_test, y_dev, y_test = train_test_split(X, y, test_size=1/3, random_state=1)
# splitting development into train (1/3) and validation (1/3)
X_train, X_val, y_train, y_val = train_test_split(X_dev, y_dev, test_size=1/2, random_state=1)

perform = []
lambdas = np.logspace(-20, 4, 33)
for lambda_ in lambdas:
    pipe_ridge = make_pipeline(PolynomialFeatures(include_bias=True), 
                               StandardScaler(),
                               Ridge(alpha=lambda_, random_state=1))
    pipe_ridge.fit(X_train, y_train)
    y_pred = pipe_ridge.predict(X_val)
    perform.append(mse(y_pred, y_val))
    
hyperparam_perform = pd.Series(perform,index=lambdas)
optimal = hyperparam_perform.nsmallest(1)    
print('Optimal lambda:', optimal.index[0])
print('Validation MSE: %.8f' % optimal.values[0])
lambda_ridge = optimal.index[0] # Sofie Add on

Optimal lambda: 0.01
Validation MSE: 0.00378513


In [39]:
#Sofie Add on
pipe_ridge = make_pipeline(PolynomialFeatures(include_bias=True),
                           StandardScaler(),
                           Ridge(alpha = lambda_ridge, random_state=1))
pipe_ridge.fit(X_dev,y_dev)
"""
errors = pipe_ridge.predict(X_test)-y_test
error_gdf = final_gdf.merge(pd.DataFrame(errors), left_index=True, right_index=True, how='left')
error_gdf.plot(column='left_share_norm_y', legend=True)
"""

print('Ridge MSE on test data:', mse(pipe_ridge.predict(X_test), y_test))

Ridge MSE on test data: 0.004365310981042533


## Lasso - cross-validation 

In [40]:
# splitting into development (2/3) and test data (1/3)
X_dev, X_test, y_dev, y_test = train_test_split(X, y, test_size=1/3, random_state=1)

kfolds = KFold(n_splits=10)

folds = list(kfolds.split(X_dev, y_dev))
# outer loop: lambdas
mselassoCV = []
lambdas = np.logspace(-6, -1, 33)
for lambda_ in lambdas:    
    # inner loop: folds
    mselassoCV_ = []    
    for train_idx, val_idx in folds:        
        # train model and compute MSE on test fold
        pipe_lassoCV = make_pipeline(PolynomialFeatures(degree=2, include_bias=True),
                                     StandardScaler(),
                                     Lasso(alpha=lambda_, random_state=1))            
        X_train, y_train = X_dev.values[train_idx], y_dev.values[train_idx]
        X_val, y_val = X_dev.values[val_idx], y_dev.values[val_idx] 
        pipe_lassoCV.fit(X_train, y_train)        
        mselassoCV_.append(mse(pipe_lassoCV.predict(X_val), y_val))    
        
    # store result    
    mselassoCV.append(mselassoCV_) 
    
# convert to DataFrame
lambdalassoCV = pd.DataFrame(mselassoCV, index=lambdas)

In [41]:
optimal = lambdalassoCV.mean(axis = 1).nsmallest(1)
"""
plot_df = pd.DataFrame(lambdalassoCV).reset_index()
plot_df['index'] = np.log10(plot_df['index'])
plot_df['x_axis'] = plot_df['index']
sns.lineplot(data=plot_df, x='x_axis', y=0)
"""

lambda_lassoCV = optimal.index[0]

In [42]:
# Sofie Add on
pipe_lassoCV = make_pipeline(PolynomialFeatures(degree=2, include_bias=True),
                                     StandardScaler(),
                                     Lasso(alpha=lambda_lassoCV, random_state=1))

pipe_lassoCV.fit(X_dev,y_dev)
print('Lasso CV MSE:', mse(pipe_lassoCV.predict(X_test), y_test))

Lasso CV MSE: 0.004517433489375399


## Ridge - cross validation

In [43]:
# splitting into development (2/3) and test data (1/3)
X_dev, X_test, y_dev, y_test = train_test_split(X, y, test_size=1/3, random_state=1)

kfolds = KFold(n_splits=10)
# kfolds = LeaveOneOut()

folds = list(kfolds.split(X_dev, y_dev))
# outer loop: lambdas
mseridgeCV = []
lambdas = np.logspace(-8, 0, 100)
for lambda_ in lambdas:    
    # inner loop: folds
    mseridgeCV_ = []    
    for train_idx, val_idx in folds:        
        # train model and compute MSE on test fold
        pipe_ridgeCV = make_pipeline(PolynomialFeatures(degree=2, include_bias=True),
                                     StandardScaler(),
                                     Ridge(alpha=lambda_, random_state=1))            
        X_train, y_train = X_dev.values[train_idx], y_dev.values[train_idx]
        X_val, y_val = X_dev.values[val_idx], y_dev.values[val_idx] 
        pipe_ridgeCV.fit(X_train, y_train)        
        mseridgeCV_.append(mse(pipe_ridgeCV.predict(X_val), y_val))    
        
    # store result    
    mseridgeCV.append(mseridgeCV_) 
    
# convert to DataFrame
lambdaridgeCV = pd.DataFrame(mseridgeCV, index=lambdas)

In [44]:
optimal = lambdaridgeCV.mean(axis=1).nsmallest(1) 
print(optimal)
"""
plot_df = pd.DataFrame(lambdaridgeCV).reset_index()
plot_df['index'] = np.log10(plot_df['index'])
plot_df['x_axis'] = plot_df['index']
sns.lineplot(data=plot_df, x='x_axis', y=0)
"""
lambda_ridgeCV = optimal.index[0] # Sofie Add on

0.001233    0.004028
dtype: float64


In [45]:
# Define preferable modal
pipe_ridgeCV = make_pipeline(PolynomialFeatures(),
                             StandardScaler(),
                             Ridge(alpha=lambda_ridgeCV, random_state=1))

pipe_ridgeCV.fit(X_dev, y_dev)
print('Ridge CV MSE:', mse(pipe_ridgeCV.predict(X_test),y_test))

Ridge CV MSE: 0.00436446379979942


## Elastic Net - cross validation

In [46]:
# compare performance
models = {'LinReg': pipe_lr, 'Lasso': pipe_lasso, 'Lasso CV': pipe_lassoCV,
         'Ridge': pipe_ridge, 'Ridge CV': pipe_ridgeCV,}

In [47]:
# splitting into development (2/3) and test data (1/3)
X_dev, X_test, y_dev, y_test = train_test_split(X, y, test_size=1/3, random_state=1)

pipe_el = make_pipeline(PolynomialFeatures(include_bias=True),StandardScaler(),ElasticNet(random_state=1))
gs = GridSearchCV(estimator=pipe_el,
                  param_grid = {'elasticnet__alpha':np.logspace(-6,2,40)*2,
                              'elasticnet__l1_ratio':np.linspace(0,1,10)},
                  scoring='neg_mean_squared_error',
                  n_jobs=4,
                  iid=False,
                  cv=10)
models['ElasticNetCV'] = gs.fit(X_dev, y_dev)

## Compare performance

In [19]:
# splitting into development (2/3) and test data (1/3)
X_dev, X_test, y_dev, y_test = train_test_split(X, y, test_size=1/3, random_state=1)
# splitting development into train (1/3) and validation (1/3)
X_train, X_val, y_train, y_val = train_test_split(X_dev, y_dev, test_size=1/2, random_state=1)

print('Validation performance'.upper())
print('Fit on train, test on val')
for name, model in models.items():
    model.fit(X_train, y_train)
    score = mse(model.predict(X_val),y_val)
    print(name, round(score, 6))

print()
print('Test performance 1'.upper())
print('Fit on train, test on test')
for name, model in models.items():
    model.fit(X_train, y_train)
    score = mse(model.predict(X_test), y_test)
    print(name, round(score, 6))

print()
print('Test performance 2'.upper())
print('Fit on dev, test on test')
for name, model in models.items():
    model.fit(X_dev, y_dev)
    score = mse(model.predict(X_test), y_test)
    print(name, round(score, 6))

VALIDATION PERFORMANCE
Fit on train, test on val
LinReg 0.004163
Lasso 0.003927
Lasso CV 0.00394
Ridge 0.003785
Ridge CV 0.003804
ElasticNetCV 0.003969

TEST PERFORMANCE 1
Fit on train, test on test
LinReg 0.004557
Lasso 0.00464
Lasso CV 0.004662
Ridge 0.004505
Ridge CV 0.004528
ElasticNetCV 0.004714

TEST PERFORMANCE 2
Fit on dev, test on test
LinReg 0.004349
Lasso 0.004512
Lasso CV 0.004517
Ridge 0.004365
Ridge CV 0.004364
ElasticNetCV 0.004509


In [20]:
# Compare on test and validation mse as well as correct prediction share.

for model in models.values():
    model.fit(X_train, y_train)

pred_lr = pipe_lr.predict(X_test)
pred_lasso = pipe_lasso.predict(X_test)
pred_lassoCV = pipe_lassoCV.predict(X_test)
pred_ridge = pipe_ridge.predict(X_test)
pred_ridgeCV = pipe_ridgeCV.predict(X_test)
pred_el = gs.predict(X_test)

models_test_df = pd.DataFrame({'LinReg': pred_lr, 'Lasso': pred_lasso, 'LassoCV':pred_lassoCV, 'Ridge': pred_ridge, 'RidgeCV': pred_ridgeCV, 'ElasticNet': pred_el, 'Left share': y_test})

# dummy for left majority
for model in models_test_df.columns:
    models_test_df[model+'_majority'] = (models_test_df[model] >0.5)*1

# correct prediction and test mse
for model in models_test_df[['LinReg', 'Lasso', 'LassoCV', 'Ridge', 'RidgeCV', 'ElasticNet']]:
    models_test_df[model+'_correct'] = (models_test_df[model+'_majority'] == models_test_df['Left share_majority'])*1

    models_test_df[model+'_testMSE'] = mse(models_test_df[model], y_test)

# validation mse
for name, model in models.items():
    model.fit(X_train, y_train)
    models_test_df[name+'_valMSE'] = mse(model.predict(X_val), y_val)

In [21]:
# make table for report
table = pd.DataFrame(models_test_df[['LinReg_correct', 'Lasso_correct', 'LassoCV_correct', 'Ridge_correct', 'RidgeCV_correct', 'ElasticNet_correct', 'LinReg_valMSE', 'Lasso_valMSE', 'Lasso CV_valMSE', 'Ridge_valMSE', 'Ridge CV_valMSE', 'ElasticNetCV_valMSE', 'LinReg_testMSE', 'Lasso_testMSE', 'LassoCV_testMSE', 'Ridge_testMSE', 'RidgeCV_testMSE', 'ElasticNet_testMSE' ]].mean())

table['correct'] = table.index.str.contains('correct')*table[0]
table['valMSE'] = table.index.str.contains('valMSE')*table[0]
table['testMSE'] = table.index.str.contains('testMSE')*table[0]

table['model'] = ['LinReg', 'Lasso', 'LassoCV', 'Ridge', 'RidgeCV', 'ElasticNetCV']*3
table = table.drop(0, axis = 1)
table = table.reset_index(drop = True)\
                .reset_index()
table = table.groupby(by = 'model')\
                .max()\
                .sort_values(by = 'index')\
                .drop('index', axis = 1)

table['Correct prediction share'] = table['correct'].round(4)
table['Validation MSE'] = (table['valMSE']*10**(2)).round(4)
table['Test MSE'] = (table['testMSE']*10**(2)).round(4)
table[['Correct prediction share', 'Validation MSE', 'Test MSE']].to_latex('models_compare.txt')

## Predicting and plotting using the cross-validated Ridge model

In [22]:
# get data from 2015
data2015 = pd.read_csv('final_data2015.csv', index_col = 0).reset_index()
data2015['left_share_norm'] = data2015['left_share']/(data2015['left_share']+data2015['right_share'])

X2015 = data2015[['karakter', 'salgspris', 'disindk', 'kriminelitet', 'lavindk', 'langledig', 'skilsmisser', 'andel_indv', 'pop_density']]
X2015[['salgspris_log', 'disindk_log', 'pop_density_log']] = np.log(X2015[['salgspris', 'disindk', 'pop_density']])
X2015 = X2015.drop(['salgspris', 'disindk', 'pop_density'],axis = 1)

y2015 = data2015['left_share_norm']


# get data from 2019
data2019 = pd.read_csv('final_data2019.csv', index_col = 0).reset_index()
data2019['left_share_norm'] = data2019['left_share']/(data2019['left_share']+data2019['right_share'])

X2019 = data2019[['karakter', 'salgspris', 'disindk', 'kriminelitet', 'lavindk', 'langledig', 'skilsmisser', 'andel_indv', 'pop_density']]
X2019[['salgspris_log', 'disindk_log', 'pop_density_log']] = np.log(X2019[['salgspris', 'disindk', 'pop_density']])
X2019 = X2019.drop(['salgspris', 'disindk', 'pop_density'],axis = 1)

y2019 = data2019['left_share_norm']

In [23]:
# fit on 2015 and make predictions for 2015
X_dev, X_test, y_dev, y_test = train_test_split(X2015, y2015, test_size = 1/3, random_state = 1)

pipe_ridgeCV.fit(X_dev, y_dev)
X2015_pred = pipe_ridgeCV.predict(X_test)
X_test['prediction'] = X2015_pred
X_test['left_majority_pred'] = (X_test['prediction'] >.5)*1
X_test['left_majority'] = (y_test>.5)*1
X_test['correct'] = (X_test['left_majority_pred'] == X_test['left_majority'])*1
X_test['error'] = X_test['prediction'] - y_test

# merge predictions to main dataset
all2015 = pd.merge(left = data2015, right = X_test[['prediction', 'left_majority_pred', 'left_majority', 'correct','error']].reset_index(), on = 'index', how = 'outer' )
all2015['test'] = (all2015['prediction'] > -100000)*1
all2015['train'] = (all2015['test'] == 0)*1

# merge to geometry
geometry2015 = gpd.read_file('geometry2015.shp').rename(columns = {'id': 'index'})
all2015 = pd.merge(left = all2015, right = geometry2015, on = 'index')\
            .set_geometry('geometry_y')

In [24]:
# develop 2019 data
data2019['prediction'] = pipe_ridgeCV.predict(X2019)
data2019['left_majority_pred'] = (data2019['prediction'] >.5)*1
data2019['left_majority'] = (data2019['left_share_norm']>.5)*1
data2019['correct'] = (data2019['left_majority_pred'] == data2019['left_majority'])*1
data2019['error'] = data2019['prediction'] - data2019['left_share_norm']

# merge to geometry
geometry2019 = gpd.read_file('geometry2019.shp').rename(columns= {'id': 'index'})
all2019 = pd.merge(left = data2019, right = geometry2019, on = 'index')\
            .set_geometry('geometry_y')

###  Prepare plots

In [25]:
data2019_dk = all2019[all2019['kommunekod'] != 400]
data2019_bornholm = all2019[all2019['kommunekod'] == 400]
data2015_dk = all2015[all2015['kommunekod'] != 400]
data2015_bornholm = all2015[all2015['kommunekod'] == 400]

In [26]:
# recode predictions higher than 100 pct. or lower than 0 pct.
def recode(n):
    if n > 1:
        return 1
    if n < 0:
        return 0
    else:
        return n

for data in [data2019_dk, data2015_bornholm, data2015_dk, data2019_bornholm]:
    data['prediction_v2'] = data['prediction'].apply(recode)

### Plot errors 

In [27]:
# with lolland
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (20,8))
cmap = 'viridis'#'RdYlGn'

vmin = min([all2019['error'].min(), all2015['error'].min()])
vmax = max([all2019['error'].max(), all2015['error'].max()])

data2015_dk.plot(column = 'train', ax = ax1, cmap = 'Greys', vmin = 0, vmax = 2)
data2015_dk.plot( column = 'error', cmap = cmap, ax = ax1, vmin = vmin, vmax =vmax)
ax1.set_axis_off()
ax1.set_title('2015', size = 25)
ax11 = plt.axes([0.315, 0.74, 0.12, 0.1])

data2015_bornholm.plot(column = 'train', ax = ax11, cmap = 'Greys', vmin = 0, vmax = 2)
data2015_bornholm.plot( column = 'error', cmap = cmap, ax = ax11, vmin = vmin, vmax =vmax)
ax11.xaxis.set_visible(False)
ax11.yaxis.set_visible(False)

data2019_dk.plot( column = 'error', cmap = cmap, ax = ax2, vmin = vmin, vmax =vmax, legend = True)
ax2.set_axis_off()
ax2.set_title('2019', size = 25)

ax21 = plt.axes([0.72, 0.74, 0.12, 0.1])
data2019_bornholm.plot(column = 'error', cmap = cmap, ax = ax21, vmin = vmin, vmax =vmax)
ax21.xaxis.set_visible(False)
ax21.yaxis.set_visible(False)

plt.savefig('errors.png')
plt.close()

In [28]:
# without lolland
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (20,8))
cmap = 'viridis'#'RdYlGn'

vmin = min([all2019[all2019['kommunekod'] != 360]['error'].min(),
           all2015[all2015['kommunekod'] != 360]['error'].min()])
vmax = max([all2019[all2019['kommunekod'] != 360]['error'].max(),
            all2015[all2019['kommunekod'] != 360]['error'].max()])

data2015_dk.plot(column = 'train', ax = ax1, cmap = 'Greys', vmin = 0, vmax = 2)
data2015_dk.plot( column = 'error', cmap = cmap, ax = ax1, vmin = vmin, vmax =vmax)
ax1.set_axis_off()
ax1.set_title('2015', size = 25)
ax11 = plt.axes([0.315, 0.74, 0.12, 0.1])

data2015_bornholm.plot(column = 'train', ax = ax11, cmap = 'Greys', vmin = 0, vmax = 2)
data2015_bornholm.plot( column = 'error', cmap = cmap, ax = ax11, vmin = vmin, vmax =vmax)
ax11.xaxis.set_visible(False)
ax11.yaxis.set_visible(False)

data2019_dk[data2019_dk['kommunekod'] != 360].plot( column = 'error', cmap = cmap, ax = ax2, vmin = vmin, vmax = vmax, legend = True)
ax2.set_axis_off()
ax2.set_title('2019', size = 25)

ax21 = plt.axes([0.72, 0.74, 0.12, 0.1])
data2019_bornholm.plot(column = 'error', cmap = cmap, ax = ax21, vmin = vmin, vmax =vmax)
ax21.xaxis.set_visible(False)
ax21.yaxis.set_visible(False)

plt.savefig('errors_wo_lolland.png')
plt.close()

### Plot predictions

In [29]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (20,8))
cmap = 'viridis'

data2019_dk.plot(column = 'left_share_norm', vmin = 0, vmax = 1, cmap = cmap, ax = ax1)
ax1.set_axis_off()
ax1.set_title('Actual outcome', size = 25)
ax11 = plt.axes([0.315, 0.74, 0.12, 0.1])

data2019_bornholm.plot( column = 'left_share_norm', vmin = 0, vmax = 1, cmap = cmap, ax = ax11)
ax11.xaxis.set_visible(False)
ax11.yaxis.set_visible(False)

data2019_dk.plot(column = 'prediction_v2', vmin = 0, vmax = 1, ax = ax2, cmap = cmap, legend = True)
ax2.set_axis_off()
ax2.set_title('Model prediction', size = 25)

ax21 = plt.axes([0.72, 0.74, 0.12, 0.1])
data2019_bornholm.plot(column = 'prediction_v2', vmin = 0, vmax = 1, ax = ax21, cmap = cmap)
ax21.xaxis.set_visible(False)
ax21.yaxis.set_visible(False)

plt.savefig('prediction.png')
plt.close()

### Plot correct predictions

In [30]:
# with lolland
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (20,8))
cmap = 'RdYlGn'

vmin = 0
vmax = 1

data2015_dk.plot(column = 'train', ax = ax1, cmap = 'Greys', vmin = 0, vmax = 2)
data2015_dk.plot( column = 'correct', cmap = cmap, ax = ax1, vmin = vmin, vmax =vmax)
ax1.set_axis_off()
ax1.set_title('2015', size = 25)
ax11 = plt.axes([0.315, 0.74, 0.12, 0.1])

data2015_bornholm.plot(column = 'train', ax = ax11, cmap = 'Greys', vmin = 0, vmax = 2)
data2015_bornholm.plot( column = 'correct', cmap = cmap, ax = ax11, vmin = vmin, vmax =vmax)
ax11.xaxis.set_visible(False)
ax11.yaxis.set_visible(False)

data2019_dk.plot( column = 'correct', cmap = cmap, ax = ax2, vmin = vmin, vmax =vmax)
ax2.set_axis_off()
ax2.set_title('2019', size = 25)

ax21 = plt.axes([0.72, 0.74, 0.12, 0.1])
data2019_bornholm.plot(column = 'correct', cmap = cmap, ax = ax21, vmin = vmin, vmax =vmax)
ax21.xaxis.set_visible(False)
ax21.yaxis.set_visible(False)

plt.savefig('correct.png')
plt.close()